In [1]:
# reload kernel after running this script
%load_ext autoreload
%autoreload 2

# import libraries
import os
from datetime import datetime

import pandas as pd
import hopsworks
from dotenv import load_dotenv

from src.paths import PARENT_DIR
from src.data import load_raw_data
from src.data import transform_raw_data_into_ts_data

In [2]:
load_dotenv(PARENT_DIR / ".env")

HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]
HOPSWORKS_PROJECT_NAME = 'taxi_demand_api'

In [3]:
start_year = 2022
end_year = datetime.now().year

print(f'Loading data from {start_year} to {end_year}...')

rides = pd.DataFrame()
for year in range(start_year, end_year + 1):
    one_year_rides = load_raw_data(year)
    rides = pd.concat([rides, one_year_rides])

Loading data from 2022 to 2023...
File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [4]:
print(f'{len(rides)=}')

len(rides)=62054825


In [5]:
ts_data = transform_raw_data_into_ts_data(rides)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
100%|██████████| 265/265 [00:09<00:00, 28.16it/s]


In [6]:
# establish connection to Hopsworks
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY,
)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/111769


In [7]:
# get feature store handle
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [9]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time series data aggregated by hour',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [10]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/111769/fs/111688/fg/117278


Uploading Dataframe: 0.00% |          | Rows 0/3669720 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/111769/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x14b9ffaf0>, None)

In [11]:
ts_data.shape

(3669720, 3)

In [12]:
rides.shape

(62054825, 3)